In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
tr=pd.read_csv('/kaggle/input/cleaning-and-data-exploration-interesting/tr.csv')

In [ ]:
ts=pd.read_csv('/kaggle/input/cleaning-and-data-exploration-interesting/ts.csv')

In [ ]:
'''Function to reduce the DF size'''
# source: https://www.kaggle.com/kernels/scriptcontent/3684066/download

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
tr=reduce_mem_usage(tr)

In [ ]:
ts=reduce_mem_usage(ts)

# Preparing datas

In [ ]:
X=tr.drop(columns=['site_id','building_id','meter_reading'])
Y=tr['meter_reading']

In [ ]:
prediction=ts.drop(columns=['site_id','building_id','row_id'])

In [ ]:
X=reduce_mem_usage(X)
prediction=reduce_mem_usage(prediction)

In [ ]:
# Split datas 
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(X,Y,test_size=0.2)

# XGBost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb=XGBRegressor()
xgb.fit(xtr,ytr)

In [ ]:
submission=pd.DataFrame({'row_id':ts['row_id']})

In [ ]:
submission['meter_reading']=xgb.predict(prediction)

In [ ]:
submission.to_csv('xgb.csv',index=False)